https://ofai.github.io/million-post-corpus/ <br> **Posts** - Headline+Body, ID_User

In [42]:
ARTICLES_MIN_LENGTH = 300
ARTICLES_PER_AUTHOR = 250

<div style="float: left;">
    Beiträge bestehen aus Headline + Body - diese werden kombiniert
    <img src="img/headline_body.png" alt="headline_body.png" style="padding-top:0.5em;"/>       
</div>
<div style="float: center; text-align:center">
    Zu kurze Beiträge - unter 300 - werden ignoriert
    <img src="img/short_body.png" alt="empty_body.png" style="padding-top:0.5em;"/>
</div>

<div style="float: left;">
    Duplikate werden ignoriert
    <img src="img/duplicate.png" alt="duplicate.png" style="padding-top:0.5em;"/>       
</div>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import sqlalchemy as sqla

con = sqlite3.connect("OneMillion/million_post_corpus/corpus.sqlite3", isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

df = pd.read_sql_query("SELECT distinct ID_User, COALESCE(Headline, '') || COALESCE(Body, '') as Beitrag from Posts", con)

df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)

df_minLength = df.loc[df['Beitrag'].apply(len) > ARTICLES_MIN_LENGTH]

eventuell auch URLS aus Beiträgen entfernen?

In [3]:
posts_per_author = df_minLength['ID_User'].value_counts()

In [4]:
df_minArticles = df_minLength.groupby('ID_User').filter(lambda x : len(x)>ARTICLES_PER_AUTHOR)

In [5]:
posts_per_author = df_minArticles['ID_User'].value_counts()

## User mit mehr als {{str(ARTICLES_PER_AUTHOR)}} individuellen Posts - (Mindestlänge {{str(ARTICLES_MIN_LENGTH)}}) : **{{str(len(posts_per_author))}}**

In [12]:
LESS_ARTICLES_PER_AUTHOR = 30

In [13]:
df_minArticles30 = df_minLength.groupby('ID_User').filter(lambda x : len(x)>LESS_ARTICLES_PER_AUTHOR)

In [15]:
less_posts_per_author = df_minArticles30['ID_User'].value_counts()

## User mit mehr als {{str(LESS_ARTICLES_PER_AUTHOR)}} individuellen Posts - (Mindestlänge {{str(ARTICLES_MIN_LENGTH)}}) : **{{str(len(less_posts_per_author))}}**

# Beispieldaten

In [43]:
import random

UserIDListe = df_minArticles.drop_duplicates(subset = ["ID_User"])['ID_User'].tolist()

ARTICLES_PER_AUTHOR_BSP = 5
AUTHORS = 5
IDS = random.sample(UserIDListe, AUTHORS)

In [44]:
author_array = []
for x in range(AUTHORS):
    author = df_minArticles[df_minArticles['ID_User'] == IDS[x]]
    author = author.reset_index()
    author = author.drop(['index'], axis=1)
    author = author.drop(['ID_User'], axis=1)
    
    author_array.append(author)


authorArticles = []
labels = []
authorId = 0

for author in author_array:
    for x in range(ARTICLES_PER_AUTHOR_BSP):
        authorArticles.append(author.loc[x, "Beitrag"])
        labels.append(authorId)
    authorId = authorId+1

#### Beispielpost von User {{IDS[0]}} - Label {{labels[:5][0]}} 

`{{authorArticles[:5][0]}}` <br>

#### Beispielpost von User {{IDS[1]}} - Label {{labels[5]}}

`{{authorArticles[5]}}` <br>

#### Beispielpost von User {{IDS[2]}} - Label {{labels[10]}} 

`{{authorArticles[10]}}` <br>

#### Beispielpost von User {{IDS[3]}} - Label {{labels[15]}} 

`{{authorArticles[15]}}` <br>

#### Beispielpost von User {{IDS[4]}} - Label {{labels[20]}} 

`{{authorArticles[20]}}` <br>